# Processing Invoices Using The Box API

In [54]:
import json
import os
import sqlite3
from box_sdk_gen import BoxClient, BoxDeveloperTokenAuth
from dotenv import load_dotenv
from google import genai
from PyPDF2 import PdfReader

load_dotenv(override=True)

True

The first step is to configure a few settings that we'll use throughout the notebook.

In [55]:
MODEL_NAME = "gemini-1.5-flash"
LOCAL_INVOICE_FOLDER = "invoices"

BOX_FOLDER_ID = os.getenv("BOX_FOLDER_ID")
BOX_DEVELOPER_TOKEN = os.getenv("BOX_DEVELOPER_TOKEN")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

We need to configure a `client` object to access the Box API. We'll use a developer token to authorize our code.

In [56]:
auth: BoxDeveloperTokenAuth = BoxDeveloperTokenAuth(token=BOX_DEVELOPER_TOKEN)
client: BoxClient = BoxClient(auth=auth)

## Downloading invoices

Let's start by downloading every invoice in Box to a local folder so we can later process them.

The first step is to return the list of all available invoices in the Box folder.

In [57]:
files = []
for item in client.folders.get_folder_items(BOX_FOLDER_ID).entries:
    files.append(item)

print(f"Found {len(files)} invoices in Box folder {BOX_FOLDER_ID}.")

Found 4 invoices in Box folder 330387637830.


Let's create the local folder where we'll store every invoice.

In [58]:
os.makedirs(LOCAL_INVOICE_FOLDER, exist_ok=True)

Now, we can download every invoice from Box to a local folder. If an invoice file already exists in the local folder, we are going to skip it.

In [59]:
for invoice in files:
    local_path = os.path.join(LOCAL_INVOICE_FOLDER, invoice.name)
    if not os.path.exists(local_path):
        print(f"Downloading {invoice.name}...")
        stream = client.downloads.download_file(invoice.id)
        with open(local_path, "wb") as f:
            for chunk in stream:
                f.write(chunk)
    else:
        print(f"{invoice.name} already exists in the local folder.")

Invoice-ETOX6AZS-0004.pdf already exists in the local folder.
Invoice-JV3QQ0DZ-0005.pdf already exists in the local folder.
Invoice-JV3QQ0DZ-0006.pdf already exists in the local folder.
Wordpress.pdf already exists in the local folder.


## Processing invoices

Once we have every invoice locally stored, we can loop and process them one by one to extract their information.

Let's start by a helper function that will help us process JSON results from Gemini.

In [60]:
def parse_json(content):
    # Try parsing as pure JSON first
    try:
        return json.loads(content)
    except json.JSONDecodeError:
        pass

    # If that fails, try to extract JSON from the text
    # Look for JSON-like content between curly braces
    import re

    json_match = re.search(r"\{.*\}", content, re.DOTALL)
    if json_match:
        try:
            return json.loads(json_match.group())
        except json.JSONDecodeError:
            pass

    # If all else fails, return the original text
    return content

Now, let's create a function that we'll extract the fields from a given invoice.

In [61]:
def extract_invoice_fields(file: str, invoice: str):
    """
    Extract data from the supplied invoice text.
    """
    print(f"Extracting data from invoice {file}...")
    client = genai.Client(api_key=GEMINI_API_KEY)

    prompt = (
        "Extract the following information from this invoice text: "
        "1. Client name "
        "2. Invoice amount "
        "3. Product name "
        "Return the result as a JSON object."
        "Use the the following keys: "
        "1. client_name (string) "
        "2. invoice_amount (float) "
        "3. product_name (string) "
        "If the information is not found, return 'null' for the corresponding key.\n"
        f"Invoice text:\n{invoice}"
    )

    try:
        response = client.models.generate_content(model=MODEL_NAME, contents=prompt)
        result = parse_json(response.candidates[0].content.parts[0].text)
        result["file"] = file
        print(json.dumps(result, indent=4))
        return result
    except Exception as e:
        print(f"Failed to extract data using Gemini. Exception:{e}")

Let's set up the database where we'll store the information of every invoice.

In [62]:
connection = sqlite3.connect("invoices.db")
cursor = connection.cursor()
cursor.execute("""
    CREATE TABLE IF NOT EXISTS invoices (
        file TEXT PRIMARY KEY UNIQUE,
        client TEXT,
        amount REAL,
        product TEXT
    )
""")
connection.commit()

Let's now process all invoices in the local folder and extract the appropriate fields. After we process an invoice, we'll update the database with all of its data.

In [63]:
cursor = connection.cursor()

for file in os.listdir(LOCAL_INVOICE_FOLDER):
    if file.endswith(".pdf"):
        print(f"\nProcessing {file}...")
        try:
            reader = PdfReader(os.path.join(LOCAL_INVOICE_FOLDER, file))
            text = ""
            for page in reader.pages:
                text += page.extract_text() or ""

            data = extract_invoice_fields(file, text)

            print(f"Updating database with invoice {data['file']}...")
            cursor.execute(
                """
                INSERT INTO invoices (file, client, amount, product)
                VALUES (?, ?, ?, ?)
                ON CONFLICT(file) DO UPDATE SET
                    client=excluded.client,
                    amount=excluded.amount,
                    product=excluded.product
                """,
                (
                    data["file"],
                    data["client_name"],
                    data["invoice_amount"],
                    data["product_name"],
                ),
            )
        except Exception as e:
            print(f"Failed to extract text from {file}. Exception: {e}")

connection.commit()


Processing Wordpress.pdf...
Extracting data from invoice Wordpress.pdf...
{
    "client_name": "Test Business",
    "invoice_amount": 93.5,
    "product_name": "Web Design",
    "file": "Wordpress.pdf"
}
Updating database with invoice Wordpress.pdf...

Processing Invoice-JV3QQ0DZ-0005.pdf...
Extracting data from invoice Invoice-JV3QQ0DZ-0005.pdf...
{
    "client_name": "Acme Inc",
    "invoice_amount": 400.0,
    "product_name": "Machine Learning School",
    "file": "Invoice-JV3QQ0DZ-0005.pdf"
}
Updating database with invoice Invoice-JV3QQ0DZ-0005.pdf...

Processing Invoice-ETOX6AZS-0004.pdf...
Extracting data from invoice Invoice-ETOX6AZS-0004.pdf...
{
    "client_name": "Santiago Valdarrama",
    "invoice_amount": 15.0,
    "product_name": "my product",
    "file": "Invoice-ETOX6AZS-0004.pdf"
}
Updating database with invoice Invoice-ETOX6AZS-0004.pdf...

Processing Invoice-JV3QQ0DZ-0006.pdf...
Extracting data from invoice Invoice-JV3QQ0DZ-0006.pdf...
{
    "client_name": "Acme Inc"

## Generating final reports

Finally, we want to generate a couple of reports with the data that we stored in the database.

In [64]:
print("\nInvoice Report")

cursor = connection.cursor()
cursor.execute("SELECT COUNT(*), SUM(amount) FROM invoices")
total_invoices, total_amount = cursor.fetchone()

print(f"* Total invoices: {total_invoices}")
print(f"* Total amount: {total_amount}")

print("\nBreakdown by client:")
cursor.execute("SELECT client, COUNT(*), SUM(amount) FROM invoices GROUP BY client")
for row in cursor.fetchall():
    client, count, amount = row
    print(f"* {client}: {count} invoices (${amount})")

connection.close()


Invoice Report
* Total invoices: 4
* Total amount: 908.5

Breakdown by client:
* Acme Inc: 2 invoices ($800.0)
* Santiago Valdarrama: 1 invoices ($15.0)
* Test Business: 1 invoices ($93.5)
